In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import pandas as pd
import time
import gc

from model import GITIII,Loss_function

In [3]:
data_dir = "../../data/BC/processed1/"

ligands_info = torch.load("/".join(data_dir.split("/")[:-2]) + "/ligands.pth")
genes = torch.load("/".join(data_dir.split("/")[:-2]) + "/genes.pth")

my_model = GITIII(genes, ligands_info, node_dim=256, edge_dim=48, num_heads=2, n_layers=1, node_dim_small=16,att_dim=8)

my_model.load_state_dict(torch.load("GRIT_best.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [4]:
print(my_model)

GITIII(
  (embeddings): Embedding(
    (cell_encoder1): FFN(
      (linear1): Linear(in_features=321, out_features=1024, bias=True)
      (linear2): Linear(in_features=1024, out_features=256, bias=True)
      (act): GELU(approximate='none')
    )
    (cell_encoder2): Embedding(29, 256)
    (distance_scaler): Sequential(
      (0): FFN(
        (linear1): Linear(in_features=5, out_features=192, bias=True)
        (linear2): Linear(in_features=192, out_features=66, bias=True)
        (act): GELU(approximate='none')
      )
      (1): Sigmoid()
    )
    (distance_encoder): FFN(
      (linear1): Linear(in_features=5, out_features=192, bias=True)
      (linear2): Linear(in_features=192, out_features=66, bias=True)
      (act): GELU(approximate='none')
    )
    (distance_embedding): Sequential(
      (0): FFN(
        (linear1): Linear(in_features=5, out_features=192, bias=True)
        (linear2): Linear(in_features=192, out_features=48, bias=True)
        (act): GELU(approximate='none')
 

In [8]:
import torch.nn as nn
class Distance_scaler(nn.Module):
    def __init__(self, main_model):
        super(Distance_scaler, self).__init__()
        self.mlp = main_model.embeddings.distance_scaler
    
    def forward(self, distances):
        distance_matrix = torch.stack([1 / (distances + 1), 1 / (torch.sqrt(distances) + 1),
                                   1 / (1 + torch.square(distances)), torch.exp(-distances),
                                   torch.exp(-torch.square(distances))], dim=-1)
        return self.mlp(distance_matrix)

my_ds=Distance_scaler(my_model)

In [16]:
x=torch.arange(1,300,20)
print(x.shape,x)
scaler=my_ds(x)
print(scaler.shape,scaler)

torch.Size([15]) tensor([  1,  21,  41,  61,  81, 101, 121, 141, 161, 181, 201, 221, 241, 261,
        281])
torch.Size([15, 66]) tensor([[0.6028, 0.2822, 0.2984, 0.3703, 0.5236, 0.5095, 0.4380, 0.4973, 0.6112,
         0.4823, 0.6475, 0.3913, 0.4575, 0.3851, 0.5054, 0.4240, 0.3584, 0.5443,
         0.4523, 0.2824, 0.3609, 0.4542, 0.5520, 0.6506, 0.3285, 0.4995, 0.5083,
         0.3667, 0.1764, 0.4253, 0.8071, 0.5259, 0.4599, 0.7754, 0.3454, 0.6425,
         0.3797, 0.6773, 0.4365, 0.4481, 0.6756, 0.5087, 0.3567, 0.6617, 0.6293,
         0.6712, 0.4318, 0.5719, 0.4166, 0.4014, 0.6257, 0.4538, 0.4466, 0.3350,
         0.4458, 0.5496, 0.4942, 0.6231, 0.6599, 0.6542, 0.5546, 0.4802, 0.5605,
         0.4468, 0.4359, 0.4640],
        [0.6159, 0.2800, 0.3929, 0.4092, 0.4630, 0.5082, 0.4365, 0.4778, 0.6047,
         0.4830, 0.6929, 0.4353, 0.4476, 0.3948, 0.5077, 0.3812, 0.3730, 0.5459,
         0.5051, 0.3321, 0.4101, 0.5161, 0.5803, 0.6726, 0.4100, 0.4644, 0.5140,
         0.4295, 0.1866, 0